### Ch7. R-CNN, SSD, YOLO를 이용한 사물탐지

##### 이 장의 주요 학습 내용
* 사물탐지 (object detection)
    - 현재까지는 이미지에 대상 1개만 존재한다고 가정하여 이 물체가 무엇인지 식별하는 것이었음
    - 하지만 이미지 내에는 대상이 1개만 존재하지는 않음
    - 예를 들어, 이미지 1개 파일 내에 개, 고양이가 있는 2개의 대상이 존재할 수 있고 그 외에 더 존재할 수도 있음
    - 하나의 이미지에 여러 대상을 식별하는 것을 '사물탐지(object detection)'이라고 함

> 사물 탐지 알고리즘의 일반적인 프레임 워크 (R-CNN, SSD, YOLO)를 살펴보면서 사물탐지에 대해 알아보자

![nn](사물탐지1.png)

* 이미지 분류와 사물 탐지의 차이
    - 지금까지 배운 이미지 분류란?
        - 이미지에 있는 (하나뿐인) 대상의 클래스를 예측하는 것이 목표
        - 입력 : 하나의 대상이 있는 이미지
        - 출력 : 클래스 레이블 (고양이, 개 등)
        - 출력 예 : 각 클래스에 속할 확률 (고양이 84%)
    - 앞으로 배울 사물 탐지란?
        - 이미지에 있는 (하나 이상의) 대상의 위치를 나타내는 박스와 해당 물체의 클래스를 예측하는 것이 목표
        - 입력 : 하나 이상의 대상이 있는 이미지
        - 각 대상의 위치를 특정하는 경계 박스 (좌표)와 해당 물체의 클래스 레이블
        - 출력 예 :
            - box1의 좌표 (x, y, w, h)와 클래스에 속할 확률
            - box2의 좌표와 클래스에 속할 확률
            - 좌표 (x, y, w, h)에서 x, y는 경계 박스의 중심점 좌표이며, w와 h는 경계 박스의 너비와 높이 
<br>
<br>
* 사물 탐지는 단순한 이미지 분류 뿐만 아니라 자율주행차 기술에서의 차량, 보행자, 도로, 장애물 등의 위치를 인식하는 것에도 활용됨

* 사물 탐지 프레임 워크의 구성 요소
    1. 영역 제안 (region proposal)
        - 이미지에서 시스템이 처리할 영역인 RoI (regions of interest)를 제안하는 딥러닝 모델/알고리즘
        - RoI는 이미지 내 물체가 존재할 것이라 예상되는 영역을 의미
        - 각각 물체 존재 확신도(objectness score)가 매겨진 많은 수의 박스 정보를 출력
        - 이 중에서 높은 물체 존재 확신도가 매겨진 박스는 추가 처리를 위해 다음 단계로 전달
    <br><br>
    2. 특징 추출 및 예측
        - 각 박스 영역의 시각적 특징이 추출됨
        - 이 특징을 평가하여 물체 존재 여부와 물체의 클래스를 판단함 (ex : 사물 분류 컴포넌트)
        - 특징 추출을 위한 '사전 학습된 합성곱 신경망'이 포함됨
        - 물체를 포함했을 가능성이 높다고 판단된 모든 영역을 신경망이 분석하며 각 영역마다 아래 두 가지 예측
            - 경계 박스 예측 : 물체를 감싸는 박스의 좌표를 예측, 경계 박스의 좌표는 튜플 (x, y, w, h)로 표현
            - 클래스 예측 : 해당 영역의 물체가 각 클래스에 속할 확률을 예측하는 '소프트 맥스 함수'
    <br><br>
    3. 비최대 억제 (non-maximum suppression, NMS)
        - 이 단계쯤이면 모델이 같은 물체에 대해 복수의 박스를 발견했을 가능성이 높음
        - 같은 물체에 대한 중복된 박스를 탐지하고 통합해서 물체 하나마다 하나의 박스만을 남도록 하는 역할
        - 예측 확률이 가장 높은 경계 박스만 남기고 나머지는 배제
        - NMS 알고리즘은 사물 탐지 프레임워크에서 거의 표준으로 사용됨
        - "신뢰 임곗값과 NMS 임곗값 등 하이퍼파라미터 조정이 필요할 수 있는 중요 단계"
    <br><br>
    4. 평가 지표
        - 이미지 분류의 정확도, 정밀도, 재현율 등과 비슷하게 사물 탐지에도 성능을 측정하는 고유의 평가 지표가 있음
        - 평균평균정밀도(mean average precision, mAP)
        - PR 곡선(precision-recall curve)
        - 중첩률(intersection over union, IoU)

![nn](사물탐지2.jpg)

* 사물 탐지 성능 평가 지표
    - 신경망 X는 Z FPS의 속도에서 mAP Y%를 달성했다.
        - X는 신경망의 종류,  Y는 평균평균정밀도,  Z는 초당 프레임 수
    - 초당 프레임 수 (frames per second, FPS) : 탐지 속도의 평가 지표
    - 평균평균정밀도 (mean Average Precision, mAP) : 신경망 예측 정밀도의 평가 지표
        - 0부터 100 사이의 값으로 나타냄, 값이 클수록 성능이 좋음
<br><br>
* mAP에 대해 알아보자
    - 중첩률 (IoU)
        - 2개의 경계 박스가 중첩되는 정도를 나타내는 값<br>
        ![nn](사물탐지3.png)<br>
        - 중첩률이 임곗값을 초과하면 해당 예측은 정답을 맞힌 것 (TP, 진양성)<br>
        - 그렇지 않으면 틀린 것 (FP, 위양성)<br><br>
    - PR 곡선
        - 재현율 = TP / (TP + FN)<br>
        - 정밀도 = TP / (TP + FP)<br>
        ![nn](사물탐지4.png)<br>
        - FN은 위음성<br><br><br>
        ![nn](사물탐지5.png)<br>
        - 재현율이 증가할 때, 정밀도가 하락하지 않는다 = 임곗값을 변화시켜도 정밀도가 하락하지 않는다 = 모델 성능이 뛰어나다
        - 성능이 낮은 모델은 재현율을 끌어올리면서 위양성의 수가 늘어남
        - 대부분 PR 곡선이 높은 정밀도로 시작해서 정밀도가 하락하는 패턴을 보임

#### 영역 기반 합성곱 신경망
##### R-CNN, Fast R-CNN, Faster R-CNN

* R-CNN (region-based convolutional neural network)
    - 모든 다중 물체 인식 알고리즘의 동작을 이해하기 위한 밑바탕이 되는 구조
    - 합성곱 신경망이 물체 인식 및 위치 특정 문제에서 대규모로 응용된 최초의 사례
    - 이후 제안된 사물 탐지 알고리즘의 기반<br>
    ![nn](사물탐지6.png)<br>
    <br>
    - R-CNN의 구송 요소 4가지
        1. RoI 추출기
            - 영역 제안 모듈
            - 선택적 탐색 알고리즘
            - 고정된 크기로 추출 : RoI의 영역 크기는 일반적으로 제각각이지만 CNN에 입력하기 위해서는 고정된 크기여야 하기 때문
        2. 특징 추출 모듈
            - 사전 학습된 합성곱 신경망 사용
            - RoI 영역을 입력해서 특징 추출
        3. 분류 모듈
            - 서포트 벡터 머신 등 기존 머신러닝 알고리즘으로 분류기를 학습한 다음 특징 추출 모듈에서 추출한 특징으로 해당 물체 분류
        4. 위치 특정 모듈
            - 경계 박스 회귀 모듈(bounding-box regressor)
            - 경계 박스 (x, y, w, h) 예측
            - x, y 경계박스의 중심점 좌표  /  w, h 경계 박스의 너비와 높이
            <br>![nn](사물탐지7.png)<br>
            <br>
            <br>
* R-CNN의 단점
    - R-CNN은 RoI 모듈을 제외한 모든 모듈은 학습이 필요함
        - 특징 추출 모듈로 사용할 합성곱 신경망 학습 필요
        - SVM 분류기 학습 필요
        - 경계 박스 회귀 모듈 학습 필요
        - 3개의 모듈을 완전히 별개로 학습 시켜야 하며, 이미 계산한 결과를 재사용 할 수 없음
    - R-CNN은 하나의 신경망으로 구성된 완결적인 시스템이 아니며 서로 독립된 여러 알고리즘이 조합되어 동작하는 시스템
        - 사물 탐지 속도가 느림
            - 이미지 한 장마다 2,000개 이상의 RoI가 제안되고 이들 영역이 모두 전체 파이프라인(CNN 특징 추출기 및 분류기)을 통과해야 하는 구조
            - RoI 하나마다 합성곱 신경망의 순방향 계산이 필요하며 그 결과를 재사용하지 못해 계산 부하가 매우 크고 속도가 느림
        - 학습 과정이 다단계로 구성
            - CNN 특징 추출기, SVM 분류기, 경계 박스 회귀 모듈의 학습을 각각 따로 진행해야 하며 복잡함
        - 학습의 공간 및 시간 복잡도가 높음
            - SVM 분류기와 경계 박스 회귀 모듈을 학습할 때 각 RoI에 추출된 특징을 디스크에 저장해야 함

* Fast R-CNN
    - 아래 두 가지 설계 수정을 통해 처리 속도와 정확도 개선
        1. CNN 특징 추출기를 맨 앞에 배치해 전체 입력 이미지로부터의 영역을 제안
            - R-CNN은 영역 제안 모듈로 파이프라인을 시작해서 그 뒤로 특징 추출 모듈을 배치
            - CNN 특징 추출기를 맨 앞에 배치하면 2,000개 이상의 중첩된 영역을 2,000개 이상의 CNN으로 처리하는 대신 전체 이미지를 하나의 CNN으로 처리할 수 있음
        2. CNN에 SVM 분류기 대신 소프트 맥스층을 추가해서 분류도 수행할 수 있게 함
            - 하나의 모듈이 특징 추출, 사물 분류의 두 가지 일을 할 수 있게 함
    ![nn](사물탐지7.png)<br>
    <br>
    - Fast R-CNN의 구성 요소
        - 특징 추출 모듈 : 신경망은 ConvNet으로 시작해 전체 이미지에서 특징 추출
        - RoI 추출기 : 선택적 탐지 알고리즘을 사용해 약 2,000개의 영역을 제안
        - RoI 풀링층
            - Fast R-CNN에서 새로 추가된 요소
            - 특징 맵에서 전결합층에 입력할 고정 크기의 영역을 추출
            - 제안된 RoI 영역에 해당하는 특징을 최대 풀링을 통해 고정 크기로 추출
        - 2개의 출력층
            - RoI의 각 클래스에 속할 확률을 출력하는 소프트 맥스층
            - 최초 제안된 RoI와의 차이를 출력하는 경계 박스 회귀층
    <br>
    - 다중 과업 손실 함수 (multi-task loss)
        - 클래스 분류
        - 경계 박스 위치 및 크기
    <br><br><br>
* Fast R-CNN의 단점
    - 성능 측정 시점에는 속도가 매우 빠름 (합성곱 신경망이 이미지 한장마다 영역 제안을 2,000개나 처리할 필요가 없기 때문)
    - 영역 제안을 생성하는 선택적 탐지 알고리즘은 여전히 속도가 느림


* Faster R-CNN
    - 영역 제안 수를 감소시켜 성능 측정 시점의 연산량을 줄여 실시간에 가까운 처리를 수행
        - 영역 제안을 위해 선택적 탐지 알고리즘 대신 "영역 제안 신경망(region proposal network, RPN)"을 도입하여 영역 제안을 학습과정에 포함
        - 예측 영역 제안은 RoI 풀링층을 사용하여 재구성, 제안된 영역 내에서 이미지를 분류하고 경계 박스의 오프셋 값을 예측에 사용
    <br><br>
    - Faster R-CNN의 구조
        - 영역 제안 신경망 (RPN)
            - 선택적 탐색 알고리즘을 대체하는 합성곱 신경망
            - 특징 추출기에서 생성한 특징 맵에서 추가 분석을 이어갈 RoI를 제안하는 역할
            - RPN의 출력은 물체 존재 확신도와 경계 박스 위치
        - Faster R-CNN : Fast R-CNN의 일반적인 구성 요소로 구성됨
            - 특징 추출기 역할을 하는 기본 신경망 : 입력 이미지에서 특징을 추출하는 일반적인 사전 학습 CNN 모델
            - RoI를 고정 크기로 변환하는 RoI 풀링층
            - 2개의 전결합층으로 나뉜 출력층
                - 클래스 분류 확률을 출력하는 소프트맥스 분류기
                - 경계 박스를 예측하는 경계 박스 회귀모듈 CNN
    <br>![nn](사물탐지9.png)<br>
    <br><br>
    - Faster R-CNN은 아래 요소로 사물 탐지 파이프라인 전체를 신경망에 내장시키는 데 성공
        1. 기본 특징 추출기
            - 입력 이미지에서 특징을 추출하는데 사용
            - 기존에는 VGG가 많이 사용되었으나 최근 ResNet이 많이 사용됨
            - ResNet은 VGG와 비교 시, 층 수가 더 많기 때문에 더 복잡한 특징을 학습할 수 있다는 장점
            - 분류와 사물 탐지에서도 마찬가지
            - 잔차 연결과 배치 정규화를 적용한 ResNet은 층수가 많은 신경망을 학습하는 데 더욱 유리함
            <br><br>
        2. 영역 제안 모듈 (RPN)
            - 사전 학습된 신경망에서 만든 특징 맵의 정보를 기초로 물체가 위치했을 법한 영역을 찾아내는 역할
            - Attention Network라고도 부름 (이 신경망이 전체 신경망의 관심사를 이미지 내 물체가 있을만한 영역으로 유도하는 역할이기 때문)
            - RPN은 2개의 층으로 구성
                - 3 X 3 전결합 합성곱층 (512채널)
                - 병렬로 배치된 2개의 1 X 1 합성곱 층
                    - 하나는 해당 영역이 물체에 포함하는지 예측하는 분류층 (배경인지 전경인지 판단하는 점수)
                    - 다른 하나는 경계 박스 회귀를 담당
            - RPN의 목표 : 해당 영역에 추가 분석이 필요한 물체가 존재하는지 여부를 판단하는 것
            - 앵커 박스 (anchor box)
                - RPN은 슬라이딩 윈도우 접근 방식을 사용해 특징 맵의 각 위치마다 k개의 영역을 생성, 이것을 앵커 박스라 표현
                    - 슬라이딩 윈도우 접근 방식이란 ? https://velog.io/@cha-suyeon/%EB%94%A5%EB%9F%AC%EB%8B%9D-Object-Detection-Sliding-Window-Convolution
                - 앵커박스는 위치가 고정되어 있어 물체의 위치를 예측하기 위한 참조점 역할
                <br>![nn](사물탐지10.jpeg)<br>
            <br><br>
        3. RoI 풀링층
            - RPN에서 제안한 영역을 추가 분석
            <br><br>
        4. 사물 분류기
        5. 경계 박스 회귀 모듈

* R-CNN ~ Fast R-CNN ~ Faster R-CNN
    - 발전 과정
    <br>![nn](사물탐지11.png)<br>
    - 각 발전 과정에서의 단점
        - R-CNN
            - RoI를 하나씩 별도로 처리하므로 처리 시간이 오래 걸림
            - 독립된 3개의 모듈을 모두 사용해야 최종 예측 가능
        - Fast R-CNN
            - 선택적 탐색 알고리즘의 느린 속도 탓에 여전히 처리 시가닝 오래 걸림
        - Faster R-CNN
            - 영역 제안에서의 시간 소모가 큼
            - 여러 시스템이 순차적으로 동작하기 때문에 전체 처리 속도가 각 시스테므이 처리 속도에 영향을 많이 받음
    <br>
    <br>
    - R-CNN의 공통적인 단점
        - 학습 과정이 번잡하고 시간이 오래 걸림
        - 학습이 여러 단계에 걸쳐 일어남
        - 추론 단계에서 신경망의 속도가 느림
    <br>
    <br>
    - R-CNN의 발전 과정은 '실시간 사물 탐지의 실현'을 위한 노력 과정

#### 다단계 탐지기와 단일 단계 탐지기 ?? <br>
##### R-CNN은 다단계 탐지기 : 2단계 탐지기 (탐지 과정이 2단계로 나뉘어짐)
        1. 선택적 탐지 알고리즘이나 RPN을 통해 RoI 추출, 가능한 경계 박스 수가 무한대이기 때문에 영역 제안 수는 상대적으로 희소
        2. 분류기는 제안된 영역만 처리
##### 단일 단계 탐지기?
        - 영역 제안 추출 단계를 건너뛰고 이미지 내 가능한 모든 위치를 대상으로 직접 탐지를 시도하는 방식
        - 처리가 빠르고 간단하지만 성능이 소폭 하락하는 경향이 있음 (정확도 ▼이나 처리속도 ▲)
        - SSD, YOLO

* 싱글샷 탐지기 (single-shot detector, SSD)
    - 합성곱층에서 위치와 클래스를 한번에 예측
    - 물체 존재 확신도는 정답과 경계 박스의 중첩률을 계산해서 예측
    <br>
    <br>
    - SSD의 구조
        - 피드포워드 합성곱 신경망 구조
            피드포워드 (feed forward) : 입력층(input layer)으로 데이터가 입력되고, 1개 이상으로 구성되는 은닉층(hidden layer)을 거쳐 마지막에 있는 출력층(output layer)으로 출력 값을 내보내는 과정
            https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=beyondlegend&logNo=221373971859
        - 여러 개의 고정 크기 경계 박스를 생성 후 각 박스에 클래스별 물체 존재 확신도를 부여한 다음 비최대 억제 알고리즘을 통해 최적 탐지 결과를 제외한 나머지를 배제
        - 다음 세 가지 요소로 구성
            1. 특징 맵을 추출하는 기본 신경망
                - 사전 학습된 신경망에서 분류기 부분 제외한 것
            2. 다중 스케일 특징층
                - 기본 신경망 뒤에 배치된 일련의 합성곱 필터
                - 점진적으로 필터 크기가 감소하며 둘 이상의 배율로 탐지를 시도함
            3. 비최대 억제
                - NMS를 적용해서 중첩되는 경계 박스를 배제하고 물체별로 경계 박스를 하나만 남김
                <br>![nn](사물탐지12.jpg)<br>
    <br><br>
    - 기본 신경망의 예측 과정
        1. SSD에는 R-CNN의 앵커 박스와 비슷한 역할을 하는 격자가 있음. 각 격자점마다 해당 앵커를 중심점으로 하는 경계 박스를 생성함. SSD에서는 이들 격자점을 프라이어(prior)라고 함
        2. SSD에서는 각 경계 박스를 별도의 이미지로 간주함. 경계 박스마다 '이 경계 박스 안에는 보트가 있는가?', 좀 더 정확히 말하자면 '이 경계 박스 안에서 보트의 특징을 뽑았는가?'를 확인함
        3. 보트의 특징을 포함하는 경계 박스를 발견했다면 해당 박스의 예측 좌표와 사물 분류 결과를 NMS층으로 전달함
        4. NMS층은 정답과 중첩률이 가장 높은 경계 박스 하나만 남기고 다른 경계 박스는 배제함
        <br>그림은 page 379 참조<br>
      <br>  
    - 다중 스케일 특징층
        - 이미지에 서로 다른 배율로 들어 있는 물체를 처리하려면 이미지를 서로 다른 크기로 전처리하고 그 결과를 합쳐서 처리할 방법이 필요함
        <br>그림은 page 380 참조<br>
    <br>
    - 비최대 억제 (NMS)
        - SSD의 순방향 계산에서 생성되는 많은 수의 경계 박스는 대부분 NMS 알고리즘을 통해 배제됨
        - 물체 하나에 경계 박스 하나만 남김

* YOLO
    - YOLOv1 ~ YOLOv2 ~ YOLOv3
    - 딥러닝으로 구현된 고속 사물 탐지 모델
    - 정확도는 R-CNN에 미치지 못하지만 빠른 처리 속도로 사물 탐지 분야에서는 인지도가 높으며 실시간 영상, 카메라 입력 처리 데모용으로 자주 사용됨
    - R-CNN과 같이 영역 제안 단계가 아예 없음
    - 그 대신 입력을 비교적 소수의 격자 형태로 분할하고 분할된 영역을 대상으로 직접 경계 박스와 사물 분류를 수행함
    - 그 결과 많은 수의 경계 박스 후보가 생성되는데 이를 다시 NMS를 사용해서 최종 예측 결과로 좁히는 방식
    <br>![nn](사물탐지13.png)<br>
    - YOLOv3에서는 소프트맥스 대신 시그모이드 함수를 사용
        - 소프트 맥스 함수를 사용하려면 경계 박스 하나가 항상 하나의 클래스에만 속한다는 가정이 필요하기 때문
        - 다중 레이블을 부여할 수 있는 모델은 정확도가 좀 더 높음
    - SSD의 앵커와 비슷하게 YOLOv3에는 이미지조각마다 세 가지 다른 배율로 탐지를 시도하는 9개의 앵커가 있음 (page 388)
        - 서로 다른 배율에 각 예측 특징 맵을 구현하여 정확도 향상에 도움
    <br>
    - YOLOv3의 전체 구조
        - 세 가지 다른 배율로 사물 탐지 수행
        <br>![nn](사물탐지14.jpg)<br>
    <br><br>
    - YOLOv3 구현해보기 - 제외

#### Ch7을 마치며...
    1. 이미지 분류는 이미지에 있는 하나의 물체 종류 또는 클래스를 예측하는 과업이다.
    2. 사물 탐지는 이미지 속 물체의 위치를 경계 박스를 통해 예측하고 그와 더불어 해당 물체의 클래스를 예측하는 과업이다.
    3. 일반적인 사물 탐지 프레임워크는 영역 제안, 특징 추출 및 예측, 비최대 억제, 평가 지표 등 네 가지 구성요소를 갖는다.
    4. 사물 탐지 알고리즘의 성능을 가늠하는 지표는 크게 속도와 정확도 두 가지다. 속도는 초당 프레임(FPS)으로 측정하고, 평균평균정밀도(mAP)로 정확도를 측정한다.
    5. 사물 탐지 시스템 중 가장 널리 알려진 것은 R-CNN 및 그 변종, SSD, YOLO 및 그 변종까지 크게 세 종류다.
    6. R-CNN은 R-CNN, Fast R-CNN, Faster R-CNN 이렇게 세 가지 변종이 있다. R-CNN과 Fast R-CNN은 RoI 제안에 선택적 탐지 알고리즘을 사용하지만 Faster R-CNN은 RoI 제안을 영역 제안 신경망으로 대체해 전체 처리 과정을 딥러닝으로 구현했다.
    7. YOLO는 YOLOv1, YOLOv2, YOLOv3까지 세 가지 버전으로 나뉜다.
    8. R-CNN은 경계 박스의 물체 존재 확신도와 물체 클래스 분류를 두 단계에 걸쳐 수행하는 다단계 탐지기다.
    9. SSD와 YOLO는 물체 존재 확신도 예측 및 물체 클래스 분류를 한 번에 마치는 단일 단계 탐지기다.
    10. 일반적으로 단일 단계 탐지기는 다단계 탐지기에 비해 정확도가 떨어지지만 속도가 훨씬 빠른 경향을 보인다.